<img src="img/automation_using_flows_header.png">

# Cross-Facility Tomography Reconstructions
<img src="img/lightsources.png">

This notebook shows how [Tomopy](https://tomopy.readthedocs.io/en/latest/) reconstructions can be automated using [Globus Flows](https://www.globus.org/globus-flows-service). The notebook aims to demonstrate how an individual flow can be used to integrate different light sources and compute facilities.



### Using the Flows GUI

The Globus Flows web app can generate a `start` page to easily invoke flows. To create this page you need to attach an input schema to the flow. This is shown at the end of the notebook.

<img src="img/input1.png">

### Monitoring the flow
Once the flow has been started you can monitor the run. The events tab shows each step of the flow and provides details regarding the input and output of each action.

Here you can see the input to the Tomopy step of the flow, showing the input values that are passed into the function to execute.

<img src="img/running.png">

# Creating a Tomopy Flow

Here we explain how the flow is defined. Running these steps will register a flow of your own that you can then run.

To run these steps you will need to install:

`pip install -U globus_sdk`

Here we create a Globus FlowsClient to securely interact with the Flows service. This will prompt you to login and paste a token into the notebook.

In [1]:
import os
import json
import time
import globus_sdk

from globus_sdk.experimental.globus_app import UserApp
from globus_sdk.scopes import GCSCollectionScopeBuilder

# Tutorial client ID
# We recommend replacing this with your own client for any production use-cases
# Create your own at developers.globus.org
CLIENT_ID = "7ca21f4a-11de-4d97-8f84-cb66f7459981"

In [2]:
my_app = UserApp("tomography-user-app", client_id=CLIENT_ID)

flows_client = globus_sdk.FlowsClient(app=my_app)

Specify the flow definition. This JSON definition is derived from the Amazon Step Functions language. States of the flow are chained together by specifying the `Next` field to construct a pipeline of operations. This flow consists of two steps:

1. TransferInputs
2. Tomocupy
3. TransferResults

The first step, TransferFiles, uses the Globus Transfer action provider. The step is given a 300s walltime and the entire input is required to be passed into the step. Static values can be used here to simplify user input.

The second step, Tomocupy, uses the Globus Compute action provider. Input is dymanically passed in as `kwargs`, which are then passed to the function to be executed. The step is given a 600s walltime and is the conclusion of the flow.

In [3]:
flow_definition = {
    "Comment": "Transfer and run Tomopy",
    "StartAt": "TransferInput",
    "States": {
        "TransferInput": {
            "Comment": "Transfer input file",
            "Type": "Action",
            "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
            "Parameters": {
                "source_endpoint_id.$": "$.input.transfer_in.source.id",
                "destination_endpoint_id.$": "$.input.transfer_in.destination.id",
                "transfer_items": [
                    {
                        "source_path.$": "$.input.transfer_in.source.path",
                        "destination_path.$": "$.input.transfer_in.destination.path",
                        "recursive": False
                    }
                ]
            },
            "ResultPath": "$.TransferInput",
            "WaitTime": 300,
            "Next": "Tomopy"
        },
        "Tomopy": {
            "Comment": "Perform Tomopy analysis",
            "Type": "Action",
            "ActionUrl": "https://compute.actions.globus.org/fxap",
            "Parameters": {
                "endpoint.$": "$.input.compute_endpoint_id",
                "function.$": "$.input.compute_function_id",
                "kwargs.$": "$.input.compute_function_kwargs"
            },
            "ResultPath": "$.TomopyOutput",
            "WaitTime": 600,
            "Next": "TransferResults"
        },
        "TransferResults": {
            "Comment": "Transfer results",
            "Type": "Action",
            "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
            "Parameters": {
                "source_endpoint_id.$": "$.input.transfer_out.source.id",
                "destination_endpoint_id.$": "$.input.transfer_out.destination.id",
                "transfer_items": [
                    {
                        "source_path.$": "$.TomopyOutput.details.result[0][1]",
                        "destination_path.=": "`$.input.transfer_out.destination.path` + `$.TomopyOutput.details.result[0][0]`",
                        "recursive": False
                    }
                ]
            },
            "ResultPath": "$.TransferFiles",
            "WaitTime": 300,
            "End": True
        },
    }
}

Register the flow. We leave the input schema blank and will later update it to support the web interface.

In [4]:
flow = flows_client.create_flow(definition=flow_definition, title="Tomopy IRI Flow", input_schema={})
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}')

Newly created flow with id:
13fb96e0-61b7-41ed-b9f4-a9bfe8e310e4


## Creating a Tomopy Function

To run Tomopy in the flow we need to define a Tomopy function and register it with Globus Compute.

**Note, this uses my personal compute endpoint, not a service account endpoint. Access to this endpoint is restricted.**

In [5]:
import globus_compute_sdk

polaris_ep = '0f44e1f7-baa5-4dfd-a2f6-73051004cfaa'
gce = globus_compute_sdk.Executor(endpoint_id=polaris_ep)

Define a function to call tomocupy.

This is a simple wrapper function that uses subprocess to invoke a templated bash command to run the tomopy cli.

In [6]:
def tomopy_wrapper(filename, command="recon", reconstruction_type="slice", 
                   save_folder="_rec", collection_path="/eagle",
                  ):
    import subprocess
    import glob
    import os
    
    cmd = f"tomopy {command} --file-name {filename} --reconstruction-type {reconstruction_type} --save-folder={save_folder}"
    res = subprocess.run(cmd.split(" "), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Get the most recent reconstruction file
    list_of_files = glob.glob(f'{save_folder}/slice_rec/*')
    recon_file = max(list_of_files, key=os.path.getctime)

    recon_filename =  os.path.basename(recon_file)
    
    # The path from a Globus endpoint, this is the output path without the collection prefix
    transfer_path = recon_file.replace(collection_path, "")
    
    return recon_filename, transfer_path,  res.returncode, res.stdout.decode("utf-8"), res.stderr.decode("utf-8")

Register the function with Compute. 

In [7]:
tomo_func = gce.client.register_function(tomopy_wrapper)

In [8]:
tomo_func

'7a88a39a-d9af-446a-bae2-bc218949829b'

### Test the function

Test the function by running directly with Globus Compute. 

In [9]:
filename = "/home/rchard/src/IRI/Tomography/phantom_projections.h5"

In [10]:
future = gce.submit(tomopy_wrapper, filename, save_folder="/eagle/APSDataAnalysis/rchard/IRI/outputs")

In [12]:
future.result()[1]

'/APSDataAnalysis/rchard/IRI/outputs/slice_rec/recon_phantom_projections-6.tiff'

## Define Facility information

To streamline switching between facility we can specify their endpoint IDs and path information.

In [13]:
# Configs for lightsources
lightsource_configs = {
    "APS": {
        "transfer_endpoint": "a17d7fac-ce06-4ede-8318-ad8dc98edd69",
        "source_path": "/Tomography/inputs/",
        "result_path": "/Tomography/outputs/",
    },
}

# Configs for Compute resources
compute_configs = {
    "ALCF-Polaris": {
        "compute_endpoint": "0f44e1f7-baa5-4dfd-a2f6-73051004cfaa",
        "transfer_endpoint": "05d2c76a-e867-4f67-aa57-76edeb0beda0", # dtn#eagle
        "collection_path": "/eagle",
        "staging_path": "/APSDataAnalysis/rchard/IRI/",
        "output_path": "/eagle/APSDataAnalysis/rchard/IRI/outputs/",
    },
    "NERSC-Perlmutter": {
        "compute_endpoint": "70f76338-f3a2-4992-9c44-f38af073bb50",
        "transfer_endpoint": "31ce9ba0-176d-45a5-add3-f37d233ba47d",
        "project_path": "",
        "staging_path": "",
        "output_path": "",
    },
}

## Running the flow
We can now specify input and start the flow.

In [14]:
def create_flow_payload(input_filename, lightsource, compute):
    flow_input = {
        "input": {
            "transfer_in": {
                "source": {
                    "id": lightsource['transfer_endpoint'],
                    "path": lightsource['source_path'] + input_filename
                },
                "destination": {
                    "id": compute['transfer_endpoint'],
                    "path": compute['staging_path'] + input_filename
                }
            },
            "transfer_out": {
                "source": {
                    "id": compute['transfer_endpoint'],
                    # Result path is provided by the function output
                },
                "destination": {
                    "id": lightsource['transfer_endpoint'],
                    "path": lightsource['result_path'],
                }
            },
            "compute_endpoint_id": compute['compute_endpoint'],
            "compute_function_id": tomo_func,
            "compute_function_kwargs": {
            "command": "recon",
            "reconstruction_type": "slice",
            "save_folder": compute['output_path'],
            "collection_path": compute['collection_path'],
            "filename": compute['collection_path'] + compute['staging_path'] + input_filename
            }
        }
    }
    return flow_input

In [15]:
lightsource = "APS"
compute_site = "ALCF-Polaris"

In [16]:
flow_input = create_flow_payload("test_data_1.h5", lightsource_configs[lightsource], compute_configs[compute_site])

Add scopes to the flow to use Mapped Collections. This is necessary if you cannot use a Guest Collection at the facility.

In [17]:
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    app=my_app,
)

In [18]:
# Get the data access scope Mapped collections
# source_transfer_endpoint = flow_input['input']['transfer_in']['source']['id']
# source_access_scope = GCSCollectionScopeBuilder(source_transfer_endpoint).make_mutable("data_access", optional=True)

dest_transfer_endpoint = flow_input['input']['transfer_in']['destination']['id']
dest_access_scope = GCSCollectionScopeBuilder(dest_transfer_endpoint).make_mutable("data_access", optional=True)

transfer_scope = globus_sdk.scopes.TransferScopes.make_mutable("all")
# transfer_scope.add_dependency(source_access_scope)
transfer_scope.add_dependency(dest_access_scope)

# Add the data access scopes as dependencies to the flow
flow_scope = specific_flow_client.scopes.make_mutable("user")
flow_scope.add_dependency(transfer_scope)

my_app.add_scope_requirements({'flow': [flow_scope]})

In [24]:
run = specific_flow_client.run_flow(
  body=flow_input,
  label="Tomopy run",
  tags=['Tomopy', 'IRI', 'example', f"Lightsource:{lightsource}", f"Compute:{compute_site}"]
)

In [25]:
run_id = run['run_id']
run_status = run['status']
print("This flow can be monitored in the Web App:")
print(f"https://app.globus.org/runs/{run_id}")
print(f"Flow run started with ID: {run_id} - Status: {run_status}")

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')
    
# Run completed
# print(json.dumps(run.data, indent=2))

This flow can be monitored in the Web App:
https://app.globus.org/runs/97b91210-cc22-4a26-a985-0feb260585c5
Flow run started with ID: 97b91210-cc22-4a26-a985-0feb260585c5 - Status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: SUCCEEDED


In [26]:
run['details']['output']['TomopyOutput']['details']['result'][0][0]

'recon_test_data_1-1.tiff'